#mongodb
1. 라이브러리 설치
2. 라이브러리 로드
3. CRUD

In [1]:
!pip install pymongo

     -------------------------------------- 382.2/382.2 kB 5.9 MB/s eta 0:00:00
     -------------------------------------- 283.7/283.7 kB 8.5 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from pymongo import MongoClient

In [3]:
client = MongoClient(
    host = 'localhost',
    port = 27017
)

In [4]:
# mongodb에 있는 데이터베이스의 목록 출력
client.list_database_names()

['admin', 'config', 'local']

In [6]:
# 데이터베이스를 선택
db = client['ubion'] #없지만 실행됨

In [7]:
client.list_database_names() #'ubion'이 생기긴 했지만 아무것도 들어있지 않아서 안 뜸

['admin', 'config', 'local']

In [8]:
# collection 접근(collection 은 sql에서 table 과 같은 의미)
collection = db['test'] #없지만 실행됨

In [9]:
# CRUD
# create -> insert()
# insert_one() /insert_many()
data = {
    'id':'test',
    'pass': '1234'
}

collection.insert_one(data)

In [10]:
collection.find_one()
# 결과가 '_id': ObjectId('643f8ae6020c96cffb922318') 이 뜻은 primary key

{'_id': ObjectId('643f8ae6020c96cffb922318'), 'id': 'test', 'pass': '1234'}

In [12]:
#mongodb의 장점. 컬럼이 다른데 생성가능
data = [
    {
        'name':'test1',
        'phone': '01011112222',
        'gender': 'male'
    },
    {
        'loc': 'seoul'
    }
]

collection.insert_many(data)

In [13]:
list(collection.find())

[{'_id': ObjectId('643f8ae6020c96cffb922318'), 'id': 'test', 'pass': '1234'},
 {'_id': ObjectId('643f8ccf020c96cffb922319'),
  'name': 'test1',
  'phone': '01011112222',
  'gender': 'male'},
 {'_id': ObjectId('643f8ccf020c96cffb92231a'), 'loc': 'seoul'}]

In [14]:
client.list_database_names()

['admin', 'config', 'local', 'ubion']